# Chaining Search Terms
---

Before we do any work, we need to import several functions from cdapython:


- `Q` and `query` which power the search
- `columns` which lets us view entity field names
- `unique_terms` which lets view entity field contents
    
We're also asking cdapython to report it's version so we can be sure we're using the one we mean to.

In [1]:
from cdapython import Q, columns, unique_terms, query
import cdapython
print(cdapython.__version__)
Q.set_host_url("http://35.192.60.10:8080/")

2022.6.22



The CDA provides a custom python tool for searching CDA data. [`Q`](usage/#q) (short for Query) offers several ways to search and filter data, and several input modes:

---

- **[Q.run()](../../../Documentation/usage/#qrun)** returns data for the specified search 
- **[Q.count()](../../../Documentation/usage/#qcounts)** returns summary information (counts) data that fit the specified search

---

## Endpoint Chaining

We're going to build on our [previous search](../BasicSearch) to see what information exists about cancers that were first diagnosed in the brain. 


In [2]:
myquery = Q('ResearchSubject.primary_diagnosis_site = "brain"')


Previously we looked at subject, research_subject, specimen and file results seperately, but we can also combine these. 

Let's say what we're really interested in is finding analysis done on specimens, so we're looking for files that belong to specimens that match our search. If we search the files endpoint directly, we'll get back all files that meet our search criteria, regardless of whether they are specimens specficially. Instead, we can chain our query to the specimen endpoint and then to the files endpoint and get the combined result:

In [3]:
myqueryspecimenfiles =  myquery.specimen.file.run()
myqueryspecimenfiles

Getting results from database

ERROR:root:
                Http Status: 500
                Error Message: Access Denied: Table broad-dsde-prod:cda_prod.all_v3_0_Files: User does not have permission to query table broad-dsde-prod:cda_prod.all_v3_0_Files.
                


Total execution time: 655 ms


We get back 50495 files that belong to specimens that meet our search criteria. As before, we can preview the results by using the `.to_dataframe()` function:

In [4]:
myqueryspecimenfiles.to_dataframe()

AttributeError: 'NoneType' object has no attribute 'to_dataframe'

<div style="background-color:#b3e5d5;color:black;padding:20px;">
<h3>Valid Endpoint Chains</h3>

Not all endpoints can be chained together. This is a restriction caused by the data itself. `diagnosis` and `treatment` information does not have files directly attached to it, instead these files are associated with the `researchsubject`, as such both "myquery.treatment.files.run()" and "myquery.diagnosis.files.run()" will fail, as there are no files to retrieve. Valid chains are:
    
<ul>
<li><b>myquery.subject.file.run:</b> This will return all the files that meet the query and that are directly tied to subject</li>
<li><b>myquery.researchsubject.file.run:</b>This will return all the files that meet the query and that are directly tied to researchsubject</li>
<li><b>myquery.specimen.file.run:</b> This will return all the files that meet the query and that are directly tied to specimen</li>
<li><b>myquery.subject.file.count.run:</b> This will return the count of files that meet the query and that are directly tied to subject</li>
<li><b>myquery.researchsubject.file.count.run:</b>This will return the count of files that meet the query and that are directly tied to researchsubject</li>
<li><b>myquery.specimen.file.count.run:</b> This will return the count of files that meet the query and that are directly tied to specimen</li>
</ul>
</div>